<a href="https://colab.research.google.com/github/solanaf/PSEN1-FAD-TF-Targets/blob/main/gimme_motifs_tf_curation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import gimmemotifs results (open/closed)

In [ ]:
import pandas as pd
# closed
motif2factors_closed = pd.read_csv('closed/combined.motif2factors.txt', sep='\t')
closed_report = pd.read_csv('closed/gimme.roc.report.txt', sep='\t')

# open
motif2factors_open = pd.read_csv('open/combined.motif2factors.txt', sep='\t')
open_report = pd.read_csv('open/gimme.roc.report.txt', sep='\t')

motif2factors_closed

,Motif,Factor,Evidence,Curated
0,GM.5.0.Sox.0001,SRY,JASPAR,Y
1,GM.5.0.Sox.0001,SOX9,Transfac,Y
2,GM.5.0.Sox.0001,Sox9,Transfac,N
3,GM.5.0.Sox.0001,SOX9,SELEX,Y
4,GM.5.0.Sox.0001,Sox9,SELEX,N
...,...,...,...,...
10274,GimmeMotifs_4,de novo,GimmeMotifs,Y
10275,GimmeMotifs_5,de novo,GimmeMotifs,Y
10276,GimmeMotifs_6,de novo,GimmeMotifs,Y
10277,GimmeMotifs_6,ZNF384,inferred (GimmeMotifs),N


In [ ]:
motif2factors_open

,Motif,Factor,Evidence,Curated
0,GM.5.0.Sox.0001,SRY,JASPAR,Y
1,GM.5.0.Sox.0001,SOX9,Transfac,Y
2,GM.5.0.Sox.0001,Sox9,Transfac,N
3,GM.5.0.Sox.0001,SOX9,SELEX,Y
4,GM.5.0.Sox.0001,Sox9,SELEX,N
...,...,...,...,...
10302,GimmeMotifs_18,FOXG1,inferred (GimmeMotifs),N
10303,GimmeMotifs_18,FOXF2,inferred (GimmeMotifs),N
10304,GimmeMotifs_18,FOXK2,inferred (GimmeMotifs),N
10305,GimmeMotifs_19,de novo,GimmeMotifs,Y


## Quality check, remove low quality motifs:
*   Curated column: (Y = high confidence, N = lower confidence)
*   Motif Name: Consider motifs with known TF assignments (avoid "Unknown").
* Drop Gimme_Motif_* if it has a p-value > 0.05 in the report file


In [ ]:
# closed
motif2factors_closed = motif2factors_closed[motif2factors_closed['Curated'] == 'Y']
motif2factors_closed = motif2factors_closed[motif2factors_closed['Evidence'] != 'Unknown']

motifs_closed = motif2factors_closed[motif2factors_closed['Motif'].str.startswith('GimmeMotifs_')]

merged_closed = motifs_closed.merge(closed_report, on='Motif', how='inner')
unsig_motifs = merged_closed[merged_closed["P-value"] >= 0.05]

motif2factors_closed = motif2factors_closed[~motif2factors_closed['Motif'].isin(unsig_motifs['Motif'])]
motif2factors_closed

,Motif,Factor,Evidence,Curated
0,GM.5.0.Sox.0001,SRY,JASPAR,Y
1,GM.5.0.Sox.0001,SOX9,Transfac,Y
3,GM.5.0.Sox.0001,SOX9,SELEX,Y
5,GM.5.0.Sox.0001,SOX13,ChIP-seq,Y
6,GM.5.0.Sox.0001,SOX9,ChIP-seq,Y
...,...,...,...,...
10268,GM.5.0.Homeodomain_POU.0026,POU3F1,ChIP-seq,Y
10270,GimmeMotifs_2,de novo,GimmeMotifs,Y
10272,GimmeMotifs_3,de novo,GimmeMotifs,Y
10275,GimmeMotifs_5,de novo,GimmeMotifs,Y


In [ ]:
# open
motif2factors_open = motif2factors_open[motif2factors_open['Curated'] == 'Y']
motif2factors_open = motif2factors_open[motif2factors_open['Evidence'] != 'Unknown']

motifs_open = motif2factors_open[motif2factors_open['Motif'].str.startswith('GimmeMotifs_')]

merged_open = motifs_open.merge(open_report, on='Motif', how='inner')
unsig_motifs = merged_open[merged_open["P-value"] >= 0.05]

motif2factors_open = motif2factors_open[~motif2factors_open['Motif'].isin(unsig_motifs['Motif'])]
motif2factors_open

,Motif,Factor,Evidence,Curated
0,GM.5.0.Sox.0001,SRY,JASPAR,Y
1,GM.5.0.Sox.0001,SOX9,Transfac,Y
3,GM.5.0.Sox.0001,SOX9,SELEX,Y
5,GM.5.0.Sox.0001,SOX13,ChIP-seq,Y
6,GM.5.0.Sox.0001,SOX9,ChIP-seq,Y
...,...,...,...,...
10288,GimmeMotifs_13,de novo,GimmeMotifs,Y
10292,GimmeMotifs_15,de novo,GimmeMotifs,Y
10294,GimmeMotifs_16,de novo,GimmeMotifs,Y
10296,GimmeMotifs_17,de novo,GimmeMotifs,Y


## Save Curated TF's to text file

In [ ]:
# save just Factor column to a .txt file
open_tfs = motif2factors_open['Factor'].tolist()
closed_tfs = motif2factors_closed['Factor'].tolist()

with open("open_tfs.txt", "w") as f:
    f.write("\n".join(motif2factors_open['Factor'].dropna().astype(str)))

with open("closed_tfs.txt", "w") as f:
    f.write("\n".join(motif2factors_closed['Factor'].dropna().astype(str)))

## See overlap with RNA-seq derived TF's
### METHOD 1
* imported open/closed integrated RNA & ATAC data
* import projected RNA TF's from viper
* checked to see which viper RNA TF's belonged in the open/closed integrated (RNA/ATAC) set by comparing the RNA TF's 'Target' to the integrated dataframe's 'gene_symbol'
* compared those resultant open & closed lists sto the  gimme motifs open & closed lists of tf's

In [ ]:
rna_closed_tfs

,TF,Target,Weight
13,AHR,CTNNA2,1
22,AHR,MAST4,1
30,AHR,RAPGEF4,1
31,AHR,RBMS1,1
47,AR,AHSG,1
...,...,...,...
13100,ZNF263,RPTOR,-1
13143,ZNF263,TCERG1L,-1
13162,ZNF263,VAV2,-1
13167,ZNF263,XXYLT1,-1


In [ ]:
closed_integrated

,gene_id,seqnames,start,end,width,strand,gene_id.y,gene_symbol,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Type
0,ENSG00000005421,chr7,95285380,95286250,871,*,55607,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed
1,ENSG00000011465,chr12,91174510,91176160,1651,*,1634,DCN,43.956834,0.957237,0.510249,1.876020,6.065260e-02,1.358119e-01,Closed
2,ENSG00000017427,chr12,102479860,102482080,2221,*,3479,IGF1,79.132196,-2.870331,0.381912,-7.515678,5.660000e-14,9.070000e-13,Closed
3,ENSG00000039139,chr5,13865254,13866010,757,*,1767,DNAH5,101.211959,1.304340,0.276030,4.725360,2.300000e-06,1.500000e-05,Closed
4,ENSG00000055609,chr7,152401940,152403940,2001,*,58508,KMT2C,3050.999434,-0.632475,0.088057,-7.182582,6.840000e-13,9.990000e-12,Closed
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
437,ENSG00000291287,chr9,40222730,40224190,1461,*,441430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed
438,ENSG00000292281,chr4,9150960,9154840,3881,*,105369250,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed
439,ENSG00000293309,chr12,97712150,97714330,2181,*,643711,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed
440,ENSG00000293412,chr13,51067160,51068170,1011,*,2974,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Closed


In [ ]:
# closed integrated data
closed_integrated = pd.read_csv("closed/Closed_Chromatin_Integrated_with_RNAseq.csv")
# open integrated data
open_integrated = pd.read_csv("open/Open_Chromatin_Integrated_with_RNAseq.csv")

# RNA-seq projected TF's, assign to open or closed based on open/closed integrated gene_symbols
rna_tfs = pd.read_csv('viper_tf_activity_results.csv')
rna_open_tfs = rna_tfs[rna_tfs['Target'].isin(open_integrated['gene_symbol'])]
rna_closed_tfs = rna_tfs[rna_tfs['Target'].isin(closed_integrated['gene_symbol'])]

In [ ]:
# Want to see where motif2factors_open['Factor'] = rna_open_tfs & other way around
final_closed_tfs = motif2factors_closed[motif2factors_closed['Factor'].isin(rna_closed_tfs['TF'])]
final_closed_tfs

final_open_tfs = motif2factors_open[motif2factors_open['Factor'].isin(rna_open_tfs['TF'])]
final_open_tfs

,Motif,Factor,Evidence,Curated
1,GM.5.0.Sox.0001,SOX9,Transfac,Y
3,GM.5.0.Sox.0001,SOX9,SELEX,Y
5,GM.5.0.Sox.0001,SOX13,ChIP-seq,Y
6,GM.5.0.Sox.0001,SOX9,ChIP-seq,Y
16,GM.5.0.Sox.0001,SOX9,TRANSFAC,Y
...,...,...,...,...
10227,GM.5.0.RFX.0018,RFX2,SELEX,Y
10247,GM.5.0.RFX.0018,RFX2,HT-SELEX,Y
10256,GM.5.0.bZIP.0103,BACH1,ChIP-seq,Y
10259,GM.5.0.C2H2_ZF.0315,RUNX1,TRANSFAC,Y


In [ ]:
len(final_closed_tfs)
unique_matched_closed_tfs = set(final_closed_tfs['Factor'])
len(set(final_closed_tfs['Factor']))

173

In [ ]:
len(final_open_tfs)
unique_matched_open_tfs = set(final_open_tfs['Factor'])
len(set(final_open_tfs['Factor']))

204

In [ ]:
with open("unique_matched_closed_tfs.txt", "w") as f:
    f.write("\n".join(unique_matched_closed_tfs))

with open("unique_matched_open_tfs.txt", "w") as f:
    f.write("\n".join(unique_matched_closed_tfs))



with open("matched_open_tfs.txt", "w") as f:
    f.write("\n".join(final_open_tfs['Factor'].dropna().astype(str)))

with open("matched_closed_tfs.txt", "w") as f:
    f.write("\n".join(final_closed_tfs['Factor'].dropna().astype(str)))


## Add matched TFs back to integrated data table by using the corresponding target according to RNA-seq Viper projection, and the gene_symbol in the integrated table

In [ ]:
atac_rna_closed_tfs = rna_closed_tfs[rna_closed_tfs['TF'].isin(final_closed_tfs['Factor'])]

filtered_closed_integrated = closed_integrated.merge(
    atac_rna_closed_tfs[['Target', 'TF']],  # Keep only relevant columns
    left_on='gene_symbol',
    right_on='Target',
    how='inner'  # Keeps only matching rows
)

filtered_closed_integrated

,gene_id,seqnames,start,end,width,strand,gene_id.y,gene_symbol,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Type,Target,TF
0,ENSG00000017427,chr12,102479860,102482080,2221,*,3479,IGF1,79.132196,-2.870331,0.381912,-7.515678,5.660000e-14,9.070000e-13,Closed,IGF1,ATF7
1,ENSG00000017427,chr12,102479860,102482080,2221,*,3479,IGF1,79.132196,-2.870331,0.381912,-7.515678,5.660000e-14,9.070000e-13,Closed,IGF1,CEBPA
2,ENSG00000017427,chr12,102479860,102482080,2221,*,3479,IGF1,79.132196,-2.870331,0.381912,-7.515678,5.660000e-14,9.070000e-13,Closed,IGF1,ESR1
3,ENSG00000017427,chr12,102479860,102482080,2221,*,3479,IGF1,79.132196,-2.870331,0.381912,-7.515678,5.660000e-14,9.070000e-13,Closed,IGF1,HNF1A
4,ENSG00000017427,chr12,102479860,102482080,2221,*,3479,IGF1,79.132196,-2.870331,0.381912,-7.515678,5.660000e-14,9.070000e-13,Closed,IGF1,JUN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
545,ENSG00000215386,chr21,16257129,16260349,3221,*,388815,MIR99AHG,3189.564115,1.155493,0.100950,11.446215,2.460000e-30,1.200000e-28,Closed,MIR99AHG,FOXP1
546,ENSG00000218336,chr4,182735960,182737230,1271,*,55714,TENM3,2189.001791,-0.060766,0.092719,-0.655377,5.122248e-01,6.705034e-01,Closed,TENM3,TEAD1
547,ENSG00000219481,chr1,16617736,16619155,1420,*,124900584,NBPF1,1113.929023,0.119568,0.107512,1.112138,2.660788e-01,4.253080e-01,Closed,NBPF1,ESR1
548,ENSG00000225830,chr10,49487360,49488590,1231,*,2074,ERCC6,959.788767,-0.361576,0.124078,-2.914093,3.567231e-03,1.202073e-02,Closed,ERCC6,HIF1A


In [ ]:
atac_rna_open_tfs = rna_open_tfs[rna_open_tfs['TF'].isin(final_open_tfs['Factor'])]

filtered_open_integrated = open_integrated.merge(
    atac_rna_open_tfs[['Target', 'TF']],  # Keep only relevant columns
    left_on='gene_symbol',
    right_on='Target',
    how='inner'  # Keeps only matching rows
)

filtered_open_integrated

,gene_id,seqnames,start,end,width,strand,gene_id.y,gene_symbol,baseMean,log2FoldChange,lfcSE,stat,pvalue,padj,Type,Target,TF
0,ENSG00000001626,chr7,117282410,117284230,1821,*,7472,CFTR,18.972606,0.615532,0.646652,0.951876,3.411601e-01,5.093295e-01,Open,CFTR,ATF1
1,ENSG00000001626,chr7,117282410,117284230,1821,*,7472,CFTR,18.972606,0.615532,0.646652,0.951876,3.411601e-01,5.093295e-01,Open,CFTR,CDX2
2,ENSG00000001626,chr7,117282410,117284230,1821,*,7472,CFTR,18.972606,0.615532,0.646652,0.951876,3.411601e-01,5.093295e-01,Open,CFTR,CEBPA
3,ENSG00000001626,chr7,117282410,117284230,1821,*,7472,CFTR,18.972606,0.615532,0.646652,0.951876,3.411601e-01,5.093295e-01,Open,CFTR,CEBPD
4,ENSG00000001626,chr7,117282410,117284230,1821,*,7472,CFTR,18.972606,0.615532,0.646652,0.951876,3.411601e-01,5.093295e-01,Open,CFTR,CTCF
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1202,ENSG00000276043,chr19,4911380,4912560,1181,*,29128,UHRF1,179.726432,2.149744,0.231852,9.272055,1.830000e-20,4.860000e-19,Open,UHRF1,CEBPA
1203,ENSG00000276043,chr19,4911380,4912560,1181,*,29128,UHRF1,179.726432,2.149744,0.231852,9.272055,1.830000e-20,4.860000e-19,Open,UHRF1,E2F1
1204,ENSG00000276043,chr19,4911380,4912560,1181,*,29128,UHRF1,179.726432,2.149744,0.231852,9.272055,1.830000e-20,4.860000e-19,Open,UHRF1,E2F7
1205,ENSG00000276043,chr19,4911380,4912560,1181,*,29128,UHRF1,179.726432,2.149744,0.231852,9.272055,1.830000e-20,4.860000e-19,Open,UHRF1,NFYB


In [ ]:
filtered_closed_integrated.to_csv('tf_closed_integrated.csv', index=False)
filtered_open_integrated.to_csv('tf_open_integrated.csv', index=False)